# Setting

Pallet Movers utilizes a hub-and-spoke system similar to many LTL carriers. Their implementation requires a truck to leave a hub and visit all of its spokes to pick up freight and return back to the hub several times per week. They have asked you to provide the best route for a truck to leave the Charlotte hub, visit all four of its spokes, and return to Charlotte with the freight it picks up. They want you to use the total distance traveled as the metric to determine which route is best. 

# Traveling Salesman Problem (TSP)

This type of problem is called the traveling salesman problem (TSP) and is a classical operations research problem. The problem is simple to state but can be very difficult to solve to optimality, especially when the number of cities (e.g., spokes) is large. For $n$ cities, there are $(n-1)!$ possible tours. When $n$ is small, you could solve the problem with the brute-force approach of calculating the total distance traveled for all possible tours.

You are instead going to try to solve the problem using integer programming.


# An Incomplete Formulation

Here is a first attempt at a formulation for the TSP. We will soon see why it is "incomplete". 


$$
\begin{array}{lrcccl}
\min & \sum\limits_{i=1}^{n} \sum\limits_{j=1}^{n} d_{ij}x_{ij} & & & \\
\textrm{subject to} & \sum\limits_{j\neq i}^{n}x_{ij} & = & 1 & \forall \textrm{   } i & \textrm{ leave each city once}\\
 & \sum\limits_{i\neq j}^{n} x_{ij} & = & 1 & \forall \textrm {   } j & \textrm{ enter each city once}\\
\textrm{where} &&&&& \\
\end{array} 
$$


$$
x_{ij} = 
    \left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if travel from city } i \text{ to city } j \\
    0, & \text{otherwise}
  \end{array}\right.
$$

$$
d_{ij} = \text{the distance between city } i \text{ and city } j
$$

What is missing from this formulation? Nothing says that you cannot have "subtours". For example, the solution with two subtours of: (1) Charlotte to Columbia to Fayetteville to Charlotte and (2) Greensboro to Spartanburg to Greensboro is feasible by these constraints. 

Your first task is to solve this problem using the given formulation and **hope** we do not encounter subtours.

# 0. Import Packages

Which packages do you want to import?

In [1]:
# import packages
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# 1. Read in Distance Data

There is a file named `distances.csv` in the folder `data` that contains the distances between each of the five cities (the hub and the four spokes) for the Charlotte Hub. Read the data into a `DataFrame` variable named `distances`, making the first column of the file the index.

In [2]:
distances = pd.read_csv('./data/distances.csv', index_col=0)
distances

,Charlotte,Greensboro,Fayetteville,Columbia,Spartanburg
Charlotte,999,93,138,93,75
Greensboro,93,999,95,183,161
Fayetteville,138,95,999,166,254
Columbia,93,183,166,999,94
Spartanburg,75,161,254,94,999


# 2. Set Up Data

How do you want to set up the data so we can use it easier in Gurobi? What things do we need?

In [3]:
routes = distances.to_dict()
routes

{'Charlotte': {'Charlotte': 999,
  'Greensboro': 93,
  'Fayetteville': 138,
  'Columbia': 93,
  'Spartanburg': 75},
 'Greensboro': {'Charlotte': 93,
  'Greensboro': 999,
  'Fayetteville': 95,
  'Columbia': 183,
  'Spartanburg': 161},
 'Fayetteville': {'Charlotte': 138,
  'Greensboro': 95,
  'Fayetteville': 999,
  'Columbia': 166,
  'Spartanburg': 254},
 'Columbia': {'Charlotte': 93,
  'Greensboro': 183,
  'Fayetteville': 166,
  'Columbia': 999,
  'Spartanburg': 94},
 'Spartanburg': {'Charlotte': 75,
  'Greensboro': 161,
  'Fayetteville': 254,
  'Columbia': 94,
  'Spartanburg': 999}}

# 3. Create the Model

In [4]:
# Create the model
m = gp.Model('pallet_movers')
m.ModelSense = GRB.MINIMIZE

Set parameter Username
Set parameter LicenseID to value 2620317
Academic license - for non-commercial use only - expires 2026-02-10


# 4. Create the Variables

In [5]:
# Create the variables
# x_ij = 1 if travel from city i to city j, 0 otherwise
x = m.addVars(routes, routes, vtype=GRB.BINARY, name='x')
m.update()
x

{('Charlotte', 'Charlotte'): <gurobi.Var x[Charlotte,Charlotte]>,
 ('Charlotte', 'Greensboro'): <gurobi.Var x[Charlotte,Greensboro]>,
 ('Charlotte', 'Fayetteville'): <gurobi.Var x[Charlotte,Fayetteville]>,
 ('Charlotte', 'Columbia'): <gurobi.Var x[Charlotte,Columbia]>,
 ('Charlotte', 'Spartanburg'): <gurobi.Var x[Charlotte,Spartanburg]>,
 ('Greensboro', 'Charlotte'): <gurobi.Var x[Greensboro,Charlotte]>,
 ('Greensboro', 'Greensboro'): <gurobi.Var x[Greensboro,Greensboro]>,
 ('Greensboro', 'Fayetteville'): <gurobi.Var x[Greensboro,Fayetteville]>,
 ('Greensboro', 'Columbia'): <gurobi.Var x[Greensboro,Columbia]>,
 ('Greensboro', 'Spartanburg'): <gurobi.Var x[Greensboro,Spartanburg]>,
 ('Fayetteville', 'Charlotte'): <gurobi.Var x[Fayetteville,Charlotte]>,
 ('Fayetteville', 'Greensboro'): <gurobi.Var x[Fayetteville,Greensboro]>,
 ('Fayetteville', 'Fayetteville'): <gurobi.Var x[Fayetteville,Fayetteville]>,
 ('Fayetteville', 'Columbia'): <gurobi.Var x[Fayetteville,Columbia]>,
 ('Fayetteville'

# 5. Set Up the Objective Function

In [6]:
# Set up objective function
m.setObjective(gp.quicksum(routes[from_city][to_city]*x[from_city,to_city] for from_city in routes
                          for to_city in routes))
m.update()
m.display()

Minimize
999.0 x[Charlotte,Charlotte] + 93.0 x[Charlotte,Greensboro]
+ 138.0 x[Charlotte,Fayetteville] + 93.0 x[Charlotte,Columbia]
+ 75.0 x[Charlotte,Spartanburg] + 93.0 x[Greensboro,Charlotte]
+ 999.0 x[Greensboro,Greensboro] + 95.0 x[Greensboro,Fayetteville]
+ 183.0 x[Greensboro,Columbia] + 161.0 x[Greensboro,Spartanburg]
+ 138.0 x[Fayetteville,Charlotte] + 95.0 x[Fayetteville,Greensboro]
+ 999.0 x[Fayetteville,Fayetteville] + 166.0 x[Fayetteville,Columbia]
+ 254.0 x[Fayetteville,Spartanburg] + 93.0 x[Columbia,Charlotte]
+ 183.0 x[Columbia,Greensboro] + 166.0 x[Columbia,Fayetteville]
+ 999.0 x[Columbia,Columbia] + 94.0 x[Columbia,Spartanburg]
+ 75.0 x[Spartanburg,Charlotte] + 161.0 x[Spartanburg,Greensboro]
+ 254.0 x[Spartanburg,Fayetteville] + 94.0 x[Spartanburg,Columbia]
+ 999.0 x[Spartanburg,Spartanburg]
Subject To
Binaries
['x[Charlotte,Charlotte]', 'x[Charlotte,Greensboro]', 'x[Charlotte,Fayetteville]',
'x[Charlotte,Columbia]', 'x[Charlotte,Spartanburg]', 'x[Greensboro,Charlott

C:\Users\mddean\AppData\Local\Temp\ipykernel_25944\1312719875.py:5: DeprecationWarning: Model.display() is deprecated
  m.display()


# 6. Create the Constraints

In [7]:
# Create the constraints
# Leave each from_city (i) exactly 1 time
for i in routes:
    m.addConstr(x.sum(i,'*') == 1, name=f'leave_{i}')

# Arrive at each to_city (j) exactly 1 time
for j in routes:
    m.addConstr(x.sum('*',j) == 1, name=f'enter_{j}')

m.update()
m.display()

Minimize
999.0 x[Charlotte,Charlotte] + 93.0 x[Charlotte,Greensboro]
+ 138.0 x[Charlotte,Fayetteville] + 93.0 x[Charlotte,Columbia]
+ 75.0 x[Charlotte,Spartanburg] + 93.0 x[Greensboro,Charlotte]
+ 999.0 x[Greensboro,Greensboro] + 95.0 x[Greensboro,Fayetteville]
+ 183.0 x[Greensboro,Columbia] + 161.0 x[Greensboro,Spartanburg]
+ 138.0 x[Fayetteville,Charlotte] + 95.0 x[Fayetteville,Greensboro]
+ 999.0 x[Fayetteville,Fayetteville] + 166.0 x[Fayetteville,Columbia]
+ 254.0 x[Fayetteville,Spartanburg] + 93.0 x[Columbia,Charlotte]
+ 183.0 x[Columbia,Greensboro] + 166.0 x[Columbia,Fayetteville]
+ 999.0 x[Columbia,Columbia] + 94.0 x[Columbia,Spartanburg]
+ 75.0 x[Spartanburg,Charlotte] + 161.0 x[Spartanburg,Greensboro]
+ 254.0 x[Spartanburg,Fayetteville] + 94.0 x[Spartanburg,Columbia]
+ 999.0 x[Spartanburg,Spartanburg]
Subject To
leave_Charlotte: x[Charlotte,Charlotte] + x[Charlotte,Greensboro] +
 x[Charlotte,Fayetteville] + x[Charlotte,Columbia] + x[Charlotte,Spartanburg] = 1
leave_Greensboro:

C:\Users\mddean\AppData\Local\Temp\ipykernel_25944\2780076475.py:11: DeprecationWarning: Model.display() is deprecated
  m.display()


# 7. Optimize

Did you find a real tour?

In [8]:
# Optimize
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 10 rows, 25 columns and 50 nonzeros
Model fingerprint: 0x686b3a4a
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1763.0000000
Presolve time: 0.01s
Presolved: 10 rows, 25 columns, 50 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)

Root relaxation: objective 4.520000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     452.0000000  452.00000

In [9]:
for v in m.getVars():
    if v.X > 0.5:
        print(f'{v.varName}={v.X}')

x[Charlotte,Columbia]=1.0
x[Greensboro,Fayetteville]=1.0
x[Fayetteville,Greensboro]=1.0
x[Columbia,Spartanburg]=1.0
x[Spartanburg,Charlotte]=1.0


# 8. Now What?

If you had subtours, then how do you suggest we get rid of  them?

In [10]:
# Add a constraint to remove subtour
m.addConstr(x['Greensboro','Fayetteville'] + x['Fayetteville','Greensboro'] <= 1, name='eliminate_G_F_subtour')
m.update()
m.display()

Minimize
999.0 x[Charlotte,Charlotte] + 93.0 x[Charlotte,Greensboro]
+ 138.0 x[Charlotte,Fayetteville] + 93.0 x[Charlotte,Columbia]
+ 75.0 x[Charlotte,Spartanburg] + 93.0 x[Greensboro,Charlotte]
+ 999.0 x[Greensboro,Greensboro] + 95.0 x[Greensboro,Fayetteville]
+ 183.0 x[Greensboro,Columbia] + 161.0 x[Greensboro,Spartanburg]
+ 138.0 x[Fayetteville,Charlotte] + 95.0 x[Fayetteville,Greensboro]
+ 999.0 x[Fayetteville,Fayetteville] + 166.0 x[Fayetteville,Columbia]
+ 254.0 x[Fayetteville,Spartanburg] + 93.0 x[Columbia,Charlotte]
+ 183.0 x[Columbia,Greensboro] + 166.0 x[Columbia,Fayetteville]
+ 999.0 x[Columbia,Columbia] + 94.0 x[Columbia,Spartanburg]
+ 75.0 x[Spartanburg,Charlotte] + 161.0 x[Spartanburg,Greensboro]
+ 254.0 x[Spartanburg,Fayetteville] + 94.0 x[Spartanburg,Columbia]
+ 999.0 x[Spartanburg,Spartanburg]
Subject To
leave_Charlotte: x[Charlotte,Charlotte] + x[Charlotte,Greensboro] +
 x[Charlotte,Fayetteville] + x[Charlotte,Columbia] + x[Charlotte,Spartanburg] = 1
leave_Greensboro:

C:\Users\mddean\AppData\Local\Temp\ipykernel_25944\1380087110.py:4: DeprecationWarning: Model.display() is deprecated
  m.display()


In [11]:
# Solve updated model
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 11 rows, 25 columns and 52 nonzeros
Model fingerprint: 0xb1113ef9
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint eliminate_G_F_subtour by 1.000000000

Found heuristic solution: objective 1763.0000000
Presolve time: 0.00s
Presolved: 11 rows, 25 columns, 53 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)

Root relaxation: objective 5.140000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec

In [12]:
for v in m.getVars():
    if v.X > 0.5:
        print(f'{v.varName}={v.X}')

x[Charlotte,Greensboro]=1.0
x[Greensboro,Fayetteville]=1.0
x[Fayetteville,Charlotte]=1.0
x[Columbia,Spartanburg]=1.0
x[Spartanburg,Columbia]=1.0


In [13]:
# Add another subtour elimination constraint
m.addConstr(x['Columbia','Spartanburg']+x['Spartanburg','Columbia'] <= 1,
            name='eliminate_C_S_subtour')
m.update()
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 12 rows, 25 columns and 54 nonzeros
Model fingerprint: 0x8f37eed4
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint eliminate_C_S_subtour by 1.000000000

Found heuristic solution: objective 1763.0000000
Presolve time: 0.00s
Presolved: 12 rows, 25 columns, 56 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)

Root relaxation: objective 5.230000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec

In [14]:
for v in m.getVars():
    if v.X > 0.5:
        print(f'{v.varName}={v.X}')

x[Charlotte,Greensboro]=1.0
x[Greensboro,Fayetteville]=1.0
x[Fayetteville,Columbia]=1.0
x[Columbia,Spartanburg]=1.0
x[Spartanburg,Charlotte]=1.0
